In [1]:
import numpy as np
import logging
import argparse
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from tqdm import tqdm
import utils
import logging


# THISneeds modification
import models.Inception_V3_finetune.net as net
from models.Inception_V3_finetune import data_loader
from evaluate import EvaluateMetrics
from models.Inception_V3_finetune import inception

import sklearn
from sklearn.metrics import accuracy_score, hamming_loss, precision_score, recall_score, f1_score


%load_ext autoreload
%autoreload 2

In [4]:
#inception_v3 = torchvision.models.inception_v3(pretrained=True)

# Instantiating the model from the torch site

In [8]:
def train(model, optimizer, loss_fn, dataloader, metrics, params, cuda_present):
    """Train the model on `num_steps` batches
    Args:
        model: (torch.nn.Module) the neural network
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches training data
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """
    
    # set model to training mode
    model.train()

    # summary for current training loop and a running average object for loss
    epoch_metric_summ = []
    loss_avg = utils.RunningAverage()
    loss_class_wts = torch.tensor(params.wts, dtype=torch.float32)
    if cuda_present:
        loss_class_wts = loss_class_wts.cuda()
    
    for i, (train_batch, labels_batch) in enumerate(dataloader):

    #for batch in range(num_batches):
        #train_batch, labels_batch = next(dataloader)
        
        #If CUDA available, move data to GPU
        if cuda_present:
           train_batch = train_batch.cuda(async=True)
           labels_batch = labels_batch.cuda(async=True)
        
        # convert to torch Variables
        train_batch, labels_batch = Variable(train_batch), Variable(labels_batch)
        
        # compute model output and loss
        prim_out, aux_out = model(train_batch)
        
        #Compute primary, Aux and total weighted loss
        loss_prim =loss_fn(prim_out, labels_batch,loss_class_wts)
        loss_aux =loss_fn(aux_out, labels_batch, loss_class_wts)
        loss = loss_prim + 0.2 * loss_aux
        
        # clear previous gradients, compute gradients of all variables wrt loss
        optimizer.zero_grad()
        loss.backward()

        # performs updates using calculated gradients
        optimizer.step()
        
        # Evaluate metrics only once in a while
        if i % params.save_summary_steps == 0:
            # extract data from torch Variable, move to cpu, convert to numpy arrays
            prim_out = prim_out.data.cpu()
            labels_batch = labels_batch.data.cpu()
            
            # compute all metrics on this batch
            summary_batch = metrics(prim_out, labels_batch, 0.5)
            
            summary_batch['loss'] = loss.item()
            epoch_metric_summ.append(summary_batch)
        
            metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in summary_batch.items())
            logging.info("- Train metrics: " + metrics_string)
            
        # update the average loss
        #loss_avg.update(loss.data())
        #if i % params.save_summary_steps == 0:
            #logging.info("batch {}, loss = {}".format(i, loss.item()))
    
    # compute epoch mean of all metrics in summary
    metrics_mean = {metric:np.mean([x[metric] for x in summary_batch]) for metric in epoch_metric_summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_mean.items())
    logging.info("- Train metrics: " + metrics_string)
        

In [9]:
def train_and_evaluate(params, train_dataloader, optimizer, loss_fn, metrics, model_dir, cuda_present):
    """Train the model and evaluate every epoch.
    Args:
        model: (torch.nn.Module) the neural network
        train_dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches training data
        val_dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches validation data
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        model_dir: (string) directory containing config, weights and log
        restore_file: (string) optional- name of file to restore from (without its extension .pth.tar)
    """

    for epoch in range(params.num_epochs):
        
        '''Do the following for every epoch'''
        # Run one epoch
        logging.info("Epoch {}/{}".format(epoch + 1, params.num_epochs))
        image_dict = train_dataloader.load_data("train", params)
        labels_dict = train_dataloader.load_labels("train", params)

        data_generator = train_dataloader.data_iterator(params, image_dict, labels_dict)
        
        # compute number of batches in one epoch (one full pass over the training set)
        train(model, optimizer, loss_fn, data_generator, metrics, params, cuda_present)

        # Evaluate for one epoch on validation set
        #val_metrics = evaluate(model, loss_fn, val_dataloader, metrics, params)
        
        '''
        
        val_acc = val_metrics['accuracy']
        is_best = val_acc>=best_val_acc
        
        
        is_best = True
        # Save weights
        utils.save_checkpoint({'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optim_dict' : optimizer.state_dict(),
                               'is_best': is_best,
                               'checkpoint': model_dir})
        
        
        
        # If best_eval, best_save_path
        if is_best:
            logging.info("- Found new best accuracy")
            best_val_acc = val_acc

            # Save best val metrics in a json file in the model directory
            best_json_path = os.path.join(model_dir, "metrics_val_best_weights.json")
            utils.save_dict_to_json(val_metrics, best_json_path)
            
        '''
        # Save latest val metrics in a json file in the model directory
        last_json_path = os.path.join(model_dir, "metrics_val_last_weights.json")
        #utils.save_dict_to_json(val_metrics, last_json_path)

# Run all the images (resized to 299 X 299 and normalized to 0 mean and stddev =1 ) through the Inception-V3 pretrained network except the final FC layer

In [6]:
cur_dir = os.getcwd()
print(cur_dir)
model_dir = "models/Inception_V3_finetune"
json_path = os.path.join(cur_dir,'models/Inception_V3_finetune//params.json')
print(json_path)

assert os.path.isfile(json_path), "No json configuration found at {}".format(json_path)
params = utils.Params(json_path)

#Set the logger
utils.set_logger(os.path.join(model_dir, 'train.log'))

logging.info("Loading the datasets")
dataloader = data_loader.Dataloader(params)

cuda_present = torch.cuda.is_available() #Boolean

if cuda_present:
    print("cuda available")
else:
    print("cuda not available")

# Move this part to net.py
inceptionV3 = inception.inception_v3() # load model from local repo

pretrained_wts = os.path.join(model_dir, 'inception_v3_google-1a9a5a14.pth')
inceptionV3.load_state_dict(torch.load(pretrained_wts))

print('done loading weights')
# Change the number of output classes from 1000 to 28

#First for the auxlogit out
aux_logit_in_ftrs = inceptionV3.AuxLogits.fc.in_features
inceptionV3.AuxLogits.fc =nn.Linear(aux_logit_in_ftrs, 28)

#Handle the primary FC layers
num_ftrs = inceptionV3.fc.in_features
inceptionV3.fc = nn.Linear(num_ftrs, 28)

#Move upto here to net.py

for param in inceptionV3.parameters():
    param.requires_grad = True
if cuda_present:
    model = inceptionV3.cuda()
else:
    model = inceptionV3

print("done transferring model to CUDA")
optimizer = optim.Adam(model.parameters(), lr = params.learning_rate)

#loss_fn = nn.BCEWithLogitsLoss()  # moving to net.py
loss_fn = net.loss_fn

metrics = net.metrics


Loading the datasets


/home/bony/Deep_Learning_Stanford_CS230/Project/Code
/home/bony/Deep_Learning_Stanford_CS230/Project/Code/models/Inception_V3_finetune//params.json
/home/bony/Deep_Learning_Stanford_CS230/Project/Data/train_299
cuda available
done loading weights
done transferring model to CUDA


In [10]:
#cuda_present = False
train_and_evaluate(params, dataloader, optimizer, loss_fn, metrics, model_dir, cuda_present)

Epoch 1/10


train
datatype is train
image id count =  31072
output image size =   299 299
31072 1942 16
mini batch index =  0


/home/bony/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bony/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/bony/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bony/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
- Train metrics: accuracy: 0.000 ; Hloss: 0.094 ; precision: 

mini batch index =  1
mini batch index =  2
mini batch index =  3
mini batch index =  4
mini batch index =  5
mini batch index =  6
mini batch index =  7
mini batch index =  8
mini batch index =  9
mini batch index =  10
mini batch index =  11
mini batch index =  12
mini batch index =  13
mini batch index =  14
mini batch index =  15
mini batch index =  16
mini batch index =  17
mini batch index =  18
mini batch index =  19
mini batch index =  20
mini batch index =  21
mini batch index =  22
mini batch index =  23
mini batch index =  24
mini batch index =  25
mini batch index =  26
mini batch index =  27
mini batch index =  28
mini batch index =  29
mini batch index =  30
mini batch index =  31
mini batch index =  32
mini batch index =  33
mini batch index =  34
mini batch index =  35
mini batch index =  36
mini batch index =  37
mini batch index =  38
mini batch index =  39
mini batch index =  40
mini batch index =  41
mini batch index =  42
mini batch index =  43
mini batch index =  

- Train metrics: accuracy: 0.000 ; Hloss: 0.065 ; precision: 0.036 ; recall: 0.004 ; F1_score: 0.007 ; loss: 0.075


...

- Train metrics: accuracy: 0.312 ; Hloss: 0.038 ; precision: 0.274 ; recall: 0.211 ; F1_score: 0.229 ; loss: 0.036


mini batch index =  1801
mini batch index =  1802
mini batch index =  1803
mini batch index =  1804
mini batch index =  1805
mini batch index =  1806
mini batch index =  1807
mini batch index =  1808
mini batch index =  1809
mini batch index =  1810
mini batch index =  1811
mini batch index =  1812
mini batch index =  1813
mini batch index =  1814
mini batch index =  1815
mini batch index =  1816
mini batch index =  1817
mini batch index =  1818
mini batch index =  1819
mini batch index =  1820
mini batch index =  1821
mini batch index =  1822
mini batch index =  1823
mini batch index =  1824
mini batch index =  1825
mini batch index =  1826
mini batch index =  1827
mini batch index =  1828
mini batch index =  1829
mini batch index =  1830
mini batch index =  1831
mini batch index =  1832
mini batch index =  1833
mini batch index =  1834
mini batch index =  1835
mini batch index =  1836
mini batch index =  1837
mini batch index =  1838
mini batch index =  1839
mini batch index =  1840


- Train metrics: accuracy: 0.188 ; Hloss: 0.049 ; precision: 0.208 ; recall: 0.140 ; F1_score: 0.158 ; loss: 0.050


mini batch index =  1901
mini batch index =  1902
mini batch index =  1903
mini batch index =  1904
mini batch index =  1905
mini batch index =  1906
mini batch index =  1907
mini batch index =  1908
mini batch index =  1909
mini batch index =  1910
mini batch index =  1911
mini batch index =  1912
mini batch index =  1913
mini batch index =  1914
mini batch index =  1915
mini batch index =  1916
mini batch index =  1917
mini batch index =  1918
mini batch index =  1919
mini batch index =  1920
mini batch index =  1921
mini batch index =  1922
mini batch index =  1923
mini batch index =  1924
mini batch index =  1925
mini batch index =  1926
mini batch index =  1927
mini batch index =  1928
mini batch index =  1929
mini batch index =  1930
mini batch index =  1931
mini batch index =  1932
mini batch index =  1933
mini batch index =  1934
mini batch index =  1935
mini batch index =  1936
mini batch index =  1937
mini batch index =  1938
mini batch index =  1939
mini batch index =  1940


KeyError: 0

In [ ]:
model.AuxLogits.fc
#model.fc.out_features

In [ ]:
mean

In [ ]:
params.cuda = torch.cuda.is_available()

inception_v3_eval = inception_v3.eval().cuda() if params.cuda else inception_v3.eval()

image_feature_dict = {}
img_count =0
t0 = time.time()

for img_id in key_list:

    #img_id = data_loader.get_random_image_id("train", params)
    input_img = data_loader.load_single_image("train", params, img_id).cuda()
    inception_v3_eval_out = inception_v3_eval(input_img)
    output = inception_v3_eval_out.cpu().numpy()
    image_feature_dict[img_id] = output
    
    if (img_count % 500 == 0):
        print(img_count, time.time() -t0)
        t0 = time.time()
    img_count += 1

print('done preprocessing image for inception net')

In [ ]:
output = inception_v3_eval_out.cpu().numpy()
output

In [ ]:
cur_dir = os.getcwd()
print(cur_dir)
json_path = os.path.join(cur_dir,'models/Inception_V3_R1/params.json')
model_dir = os.path.join(cur_dir, 'models/Inception_V3_R1')
params = utils.Params(json_path)
data_loader = Dataloader(params)
labels_dict = data_loader.load_labels("train", params)

In [ ]:
# Shuffle the images for training 
key_list = []
for key in image_feature_dict:
    key_list.append(key)
np.random.shuffle(key_list)
print(key_list[0], key_list[20])



# Define some parameters
image_count = len(labels_dict)
minibatch_size = params.batch_size
num_epochs = params.num_epochs
features_in = params.features_in
channels = 3    

num_batches = (image_count +1) // params.batch_size

print("image count = ", image_count)
print("minibatch_size  = ", minibatch_size)
print("num_epochs = ", num_epochs)
print("features_in  = ", features_in)
print("num_batches  = ", num_batches)
print("learning rate = ", params.learning_rate)

batch_data = torch.zeros([minibatch_size, features_in], dtype=torch.float32)
batch_labels = torch.zeros([minibatch_size, 28], dtype=torch.float32)

# Check if GPU is cuda capable
cuda_present = torch.cuda.is_available()

#Instantiate the model, loss fn, Optimizer
if cuda_present:
    model = net.Net(params).cuda()
else:
    model = net.Net(params)
optimizer = optim.Adam(model.parameters(), lr= params.learning_rate)
criterion = nn.BCELoss()

In [ ]:
#Train the model over a few epochs
for epoch in range(num_epochs):
    #np.random.shuffle(key_list)
    print("\n******************\n Epoch = ", epoch,"\n")
    for j in range(num_batches):
        mini_batch_index_start = j*minibatch_size
        mini_batch_index_end = (j+1)*minibatch_size
        batch_image_index = key_list[mini_batch_index_start:mini_batch_index_end ]
        #print(j, mini_batch_index_start)
        
        # Generate mini-batch data and labels
        for k in range(minibatch_size):
            img_id = key_list[mini_batch_index_start + k]
            batch_data[k,:] = torch.tensor(image_feature_dict[img_id], dtype= torch.float32)
            labels = torch.tensor(labels_dict[img_id], dtype= torch.float32)
            batch_labels[k, :] = labels
        
        #print(img_id)
        if cuda_present:
            batch_data = batch_data.cuda()
            batch_labels = batch_labels.cuda()
            
        # clear previous gradients
        optimizer.zero_grad()

        #forward propogation
        output_batch = model.forward(batch_data)

        #calculate the loss
        loss = criterion(output_batch,batch_labels)
        
        # backward propogation
        loss.backward()
        
        # performs parameter updates using calculated gradients
        optimizer.step()
        
        #cost = loss.item()
        #print(cost)
        # print statistics
        
        '''
        for param in model.fc1.parameters():
            for k in range(len(param.size())):
                print(param[k])
        ''' 
        if ( (j%200 ==0) or ((j+1)% num_batches ==0) ):
            cost = loss.item()
            #cost = cost.cpu()
            print('Batch #:' + str(j) + ", training cost = " + str(cost))
key_list[0] 
labels_dict[img_id].shape

In [ ]:
state = {'epoch':100, 'state_dict':model.state_dict(), 'optim_dict':optimizer.state_dict()}

In [ ]:
file_path = os.path.join(cur_dir, 'models/Inception_V3_R1/epoch100.pth.tar')
torch.save(state, file_path)

In [ ]:


inception_v3_eval = inception_v3.eval()
image_feature_dict = {}
image_dict = data_loader.load_data("train", params)
img_count = 0
t0 = time.time()
for img_id in image_dict:
    if (img_count %50 == 0):
        t1 = time.time()
        print("image count = ", img_count, t1-t0)
        t0 = t1
    input_img = data_loader.load_single_image("train", params, img_id)
    inception_v3_eval_out = inception_v3_eval(input_img)
    image_feature_dict[img_id] = inception_v3_eval_out

    img_count += 1

print("Finished evaluation, next step is to save the data to file")


key_list = []
for item in image_feature_dict:
    key_list.append(item)
f = open("image_resize299_inception_v3_feature.txt", "w")
for item in key_list:
    val = str(image_feature_dict[item].tolist())
    f.write(item+":"+val+"\n")
f.close()

In [ ]:
import seaborn as sns
import matplotlib as mpl

In [ ]:
# Target labels
LABEL_MAP = {
    0: "Nucleoplasm",
    1: "Nuclear membrane",
    2: "Nucleoli",
    3: "Nucleoli fibrillar center",   
    4: "Nuclear speckles",
    5: "Nuclear bodies",
    6: "Endoplasmic reticulum",
    7: "Golgi apparatus",
    8: "Peroxisomes",
    9:  "Endosomes",
    10: "Lysosomes",
    11: "Intermediate filaments", 
    12: "Actin filaments",
    13: "Focal adhesion sites",
    14: "Microtubules",
    15: "Microtubule ends",
    16: "Cytokinetic bridge",
    17: "Mitotic spindle",
    18: "Microtubule organizing center",  
    19: "Centrosome",
    20: "Lipid droplets",
    21: "Plasma membrane",
    22: "Cell junctions",
    23: "Mitochondria",
    24: "Aggresome",
    25: "Cytosol",
    26: "Cytoplasmic bodies",
    27: "Rods & rings"
}

# Defines
DIM_HEIGHT = DIM_WIDTH = 512
N_CHANNELS = 4

label = np.zeros((28))
for j in range(len(labels_dict)):
    item = key_list[j]
    tgt_label = labels_dict[item]
    label += tgt_label
    
print(label)
xx = list(LABEL_MAP.values())

plt.figure(figsize=(15, 7))
mpl.style.use('seaborn-darkgrid')
sns.barplot(x = label, y = xx)
plt.title('Target Occurences', fontsize=16)
plt.show()


In [ ]:
params.train_data_path

# Instantiating the model and initializing the weights from my laptop, downloaded from the torch sitem the local machine. Don't need this now

In [ ]:
from models.VGG19_1.vgg import VGG
#from models import vgg
vgg19_bn = VGG.vgg19_bn(pretrained=False)
curdir = os.getcwd()
vgg_dict_path = os.path.join(curdir, "models/VGG19_1/vgg19_bn-c79401a0.pth")
model_vgg19_bn.load_state_dict(torch.load(vgg_dict_path))

for param in model_vgg19_bn.parameters():
    param.requires_grad = False

In [ ]:
for x in model_vgg19_bn.classifier.children():
    print(x)
    for y in x.parameters():
        print(y.requires_grad)
for x in model_vgg19_bn.modules():
    print(x.state_dict().keys())
import json
cur_dir = os.getcwd()
print(cur_dir)
json_path = os.path.join(cur_dir,'models\params.json')
f = open(json_path)
params = json.load(f)
#params = utils.Params(json_path)
data_loader = Dataloader(params)

In [ ]:
torch.cuda.get_device_name(0)

In [3]:
import sklearn
from sklearn.metrics import accuracy_score, hamming_loss, precision_score, recall_score, f1_score

In [4]:
A1 = torch.tensor([[1,0,0],[0,1,1]],  dtype=torch.float32)
B1 = torch.tensor([[1,0,0],[0,1,1]], dtype=torch.float32)

In [8]:
import sklearn
from sklearn.metrics import accuracy_score, hamming_loss, precision_score, recall_score, f1_score
metrics = net.accuracy(A1, B1, 0.5)

In [10]:
metrics

{'F1_score': 1.0,
 'Hloss': 0.0,
 'accuracy': 1.0,
 'precision': 1.0,
 'recall': 1.0}

In [11]:
aa = metrics.items()

In [12]:
aa

dict_items([('accuracy', 1.0), ('Hloss', 0.0), ('precision', 1.0), ('recall', 1.0), ('F1_score', 1.0)])

In [14]:
outstr = " ; ".join("{}: {:05.3f}".format(k,v) for k,v in aa)

In [15]:
outstr

'accuracy: 1.000 ; Hloss: 0.000 ; precision: 1.000 ; recall: 1.000 ; F1_score: 1.000'